**Proyecto 5**

**PROCESAMIENTO DE LENGUAJE NATURAL USANDO REDES NEURONALES RECURRENTES.**

1. Para empezar impórtamos las librerias correspondientes.

In [3]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense


#LIBRERIAS PARA LA OPTENCION DE DATA
import nltk
from nltk.corpus import stopwords
from nltk.corpus import cess_esp

2.Seguidamente descargamos la data a utlizar, en este caso descargaremos el corpus cess_esp proporcionado por la libreria nltk el cual constituye una recopilacion de noticias de diferentes diarios de espana

referencia: https://clic.ub.edu/corpus/es

De este corpus lo limitamos a 200 titulares que son ideas largas. Este numero se puede aumentar para mejorar las respuestas del modelo pero significaria un entrenamiento mas largo.

A esta data le aplicamos filtros, como quitarle los signos de puntuacion, separar palabras y quitar caracteres ilegibles

In [4]:
nltk.download('cess_esp')
nltk.download('stopwords')
# Obtenemos oraciones del corpus CESS-ECE2
sentences = cess_esp.sents()
corpus = [] #variables que contendra la data final

stop_words = set(stopwords.words('spanish'))
# filtramos las oraciones
for i in range(200):
    # Filtrar palabras de parada y signos de puntuación
    filtered_tokens = [word.lower() for word in sentences[i] if (word.isalpha())]
    # Agregar la oración procesada a la lista
    corpus.append(filtered_tokens)
    print(corpus[i])

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['el', 'grupo', 'estatal', 'edf', 'anunció', 'hoy', 'jueves', 'la', 'compra', 'del', 'de', 'la', 'empresa', 'mexicana', 'eaa', 'creada', 'por', 'el', 'japonés', 'para', 'una', 'central', 'de', 'gas', 'de', 'megavatios']
['una', 'portavoz', 'de', 'edf', 'explicó', 'a', 'efe', 'que', 'el', 'proyecto', 'para', 'la', 'construcción', 'de', 'al', 'norte', 'de', 'tampico', 'prevé', 'la', 'utilización', 'de', 'gas', 'natural', 'como', 'combustible', 'principal', 'en', 'una', 'central', 'de', 'ciclo', 'combinado', 'que', 'debe', 'empezar', 'a', 'funcionar', 'en']
['la', 'electricidad', 'producida', 'pasará', 'a', 'la', 'red', 'eléctrica', 'pública', 'de', 'méxico', 'un', 'acuerdo', 'de', 'venta', 'de', 'energía', 'de', 'eaa', 'con', 'la', 'cfe', 'por', 'una', 'duración', 'de', 'años']
['edf', 'que', 'no', 'quiso', 'revelar', 'cuánto', 'pagó', 'por', 'su', 'participación', 'mayoritaria', 'en', 'eaa', 'intervendrá', 'como', 'asistente', 'en', 'la', 'construcción', 'de', 'y', 'posteriormente', 'se

3. Aplicamos tokenizacion y preparamos las secuencias de entrada, la secuencia variara segun el ancho de la linea

In [5]:
# Tokenización
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Crear secuencias de entrada y salida
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Calculamos el tamano de la secuencia maxima
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Crear X y y
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = to_categorical(y, num_classes=total_words)

4. Creamos el modelo, uasaremos un vector de entrada "embedding" de 50 espacios para optimizar el input de entrada, colocamos 160 neuronas LSTM y una capa de salida de input de la maxima palabra de la data

In [6]:
# Crear modelo
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=max_sequence_length-1))
model.add(LSTM(units=160))
model.add(Dense(units=total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

5. Entrenamos el modelo, fijamos el contador en 100 epocas y un batch de 32 para evitar la memorizacion

In [7]:
# Entrenar modelo
model.fit(X, y, epochs=100, verbose='auto', batch_size=32)

Epoch 1/100
174/174 [==============================] - 28s 142ms/step - loss: 6.5947 - accuracy: 0.0585
Epoch 2/100
174/174 [==============================] - 24s 137ms/step - loss: 6.0145 - accuracy: 0.0693
Epoch 3/100
174/174 [==============================] - 25s 145ms/step - loss: 5.8619 - accuracy: 0.0812
Epoch 4/100
174/174 [==============================] - 25s 142ms/step - loss: 5.7391 - accuracy: 0.0866
Epoch 5/100
174/174 [==============================] - 23s 130ms/step - loss: 5.6135 - accuracy: 0.0906
Epoch 6/100
174/174 [==============================] - 27s 153ms/step - loss: 5.4773 - accuracy: 0.0978
Epoch 7/100
174/174 [==============================] - 24s 140ms/step - loss: 5.3321 - accuracy: 0.1045
Epoch 8/100
174/174 [==============================] - 24s 139ms/step - loss: 5.1677 - accuracy: 0.1144
Epoch 9/100
174/174 [==============================] - 25s 144ms/step - loss: 4.9947 - accuracy: 0.1281
Epoch 10/100
174/174 [==============================] - 22s 127m

5.Creamos una funcion para generar el texto, para ver como nos responde la red a una palabra en concreto, tambien le podemos especificar el numero de palabras que queremos que la red nos regrese

ejemplo puesto: para la palabra "por la" completame un texto de 20 palabras

ajustar next_words y seed_text

In [10]:
# Función para generar texto
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)           # elegimos la mejor respuesta del modelo
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generar texto de prueba
seed_text = "por la"
generated_text = generate_text(seed_text, next_words=20, model=model, max_sequence_length=max_sequence_length)
print(generated_text)

1/1 [==============================] - 0s 34ms/step
por la reunión la mayoría de los presentes manifestaron su apoyo a lieberman que aseguró que su compromiso los derechos civiles comienza


6. Vemos que la presicion al generar texto es del 98%

In [11]:
loss, accuracy = model.evaluate(X, y)
print(f'\nModel Accuracy: {accuracy * 100:.2f}%')

174/174 [==============================] - 8s 40ms/step - loss: 0.0550 - accuracy: 0.9818

Model Accuracy: 98.18%
